# Overview

In this tutorial, we'll use Feast to inject documents and structured data (i.e., features) into the context of an LLM (Large Language Model) to power a RAG Application (Retrieval Augmented Generation).

Feast solves several common issues in this flow:
1. **Online retrieval:** At inference time, LLMs often need access to data that isn't readily 
   available and needs to be precomputed from other data sources.
   * Feast manages deployment to a variety of online stores (e.g. Milvus, DynamoDB, Redis, Google Cloud Datastore) and 
     ensures necessary features are consistently _available_ and _freshly computed_ at inference time.
2. **Vector Search:** Feast has built support for vector similarity search that is easily configured declaritively so users can focus on their application.
3. **Richer structured data:** Along with vector search, users can query standard structured fields to inject into the LLM context for better user experiences.
4. **Feature/Context and versioning:** Different teams within an organization are often unable to reuse 
   data across projects and services, resulting in duplicate application logic. Models have data dependencies that need 
   to be versioned, for example when running A/B tests on model/prompt versions.
   * Feast enables discovery of and collaboration on previously used documents, features, and enables versioning of sets of 
     data.

We will:
1. Deploy a local feature store with a **Parquet file offline store** and **Sqlite online store**.
2. Write/materialize the data (i.e., feature values) from the offline store (a parquet file) into the online store (Sqlite).
3. Serve the features using the Feast SDK
4. Inject the document into the LLM's context to answer questions

In [1]:
%%sh
pip install feast -U -q
echo "Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded."

Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded.


**Reminder**: Please restart your runtime after installing Feast (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded.

## Step 2: Create a feature repository

A feature repository is a directory that contains the configuration of the feature store and individual features. This configuration is written as code (Python/YAML) and it's highly recommended that teams track it centrally using git. See [Feature Repository](https://docs.feast.dev/reference/feature-repository) for a detailed explanation of feature repositories.

The easiest way to create a new feature repository to use the `feast init` command. For this demo, you **do not** need to initialize a feast repo.


### Demo data scenario 
- We data from Wikipedia about states that we have embedded into sentence embeddings to be used for vector retrieval in a RAG application.
- We want to generate predictions for driver satisfaction for the rest of the users so we can reach out to potentially dissatisfied users.

In [1]:
import feast

### Step 2a: Inspecting the feature repository

Let's take a look at the demo repo itself. It breaks down into


* `data/` contains raw demo parquet data
* `example_repo.py` contains demo feature definitions
* `feature_store.yaml` contains a demo setup configuring where data sources are
* `test_workflow.py` showcases how to run all key Feast commands, including defining, retrieving, and pushing features.
   * You can run this with `python test_workflow.py`.

In [2]:
%cd feature_repo/
!ls -R

/Users/farceo/dev/feast/examples/rag-docling/feature_repo
__init__.py        data               feature_store.yaml
__pycache__        example_repo.py    test_workflow.py

./__pycache__:
example_repo.cpython-311.pyc

./data:
docling_samples.parquet


/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Step 2b: Inspecting the project configuration
Let's inspect the setup of the project in `feature_store.yaml`. 

The key line defining the overall architecture of the feature store is the **provider**. 

The provider value sets default offline and online stores. 
* The offline store provides the compute layer to process historical data (for generating training data & feature 
  values for serving). 
* The online store is a low latency store of the latest feature values (for powering real-time inference).

Valid values for `provider` in `feature_store.yaml` are:

* local: use file source with Milvus Lite
* gcp: use BigQuery/Snowflake with Google Cloud Datastore/Redis
* aws: use Redshift/Snowflake with DynamoDB/Redis

Note that there are many other offline / online stores Feast works with, including Azure, Hive, Trino, and PostgreSQL via community plugins. See https://docs.feast.dev/roadmap for all supported connectors.

A custom setup can also be made by following [Customizing Feast](https://docs.feast.dev/v/master/how-to-guides/customizing-feast)

In [3]:
!pygmentize feature_store.yaml

project: rag
provider: local
registry: data/registry.db
online_store:
  type: milvus
  path: data/online_store.db
  vector_enabled: true
  embedding_dim: 384
  index_type: "IVF_FLAT"


offline_store:
  type: file
entity_key_serialization_version: 3
# By default, no_auth for authentication and authorization, other possible values kubernetes and oidc. Refer the documentation for more details.
auth:
    type: no_auth


### Inspecting the raw data

The raw feature data we have in this demo is stored in a local parquet file. The dataset Wikipedia summaries of diferent cities.

In [4]:
import pandas as pd 

df = pd.read_parquet("./data/docling_samples.parquet")
df['chunk_embedding'] = df['chunk_embedding'].apply(lambda x: x.tolist())
embedding_length = len(df['chunk_embedding'][0])
print(f'embedding length = {embedding_length}')
# can't store long text fields 
df.drop(['full_document_markdown', 'bytes'], axis=1, inplace=True)

embedding length = 384


In [5]:
from IPython.display import display

display(df.head())

,file_name,raw_chunk_markdown,chunk_embedding,chunk_id,created
0,2203.01017v2,"Ahmed Nassar, Nikolaos Livathinos, Maksym Lysa...","[-0.056879762560129166, 0.01667858101427555, -...",6385912fa27a8dd602cea2afaa3ecc9a27229ebd508661...,2025-03-02 06:25:05.958118
1,2203.01017v2,a. Picture of a table:\nTables organize valuab...,"[0.050771258771419525, -0.0055733839981257915,...",ea0a2fb2f01aef67673594966c17092acfffc147b71a0a...,2025-03-02 06:25:05.958118
2,2203.01017v2,a. Picture of a table:\ncomplex column/row-hea...,"[-0.05088765174150467, 0.05101901665329933, -0...",0de91a3e11fdad41895df9da8e23aaede5fa3673b5a3e1...,2025-03-02 06:25:05.958118
3,2203.01017v2,a. Picture of a table:\nmodel. The latter impr...,"[0.011835305020213127, -0.09409898519515991, 0...",3e4296ff2bbe2bfde9a72a9594245eb923e9e3f44c239f...,2025-03-02 06:25:05.958118
4,2203.01017v2,a. Picture of a table:\nwe can obtain the cont...,"[-0.0068757119588553905, 0.006624480709433556,...",85d4ddd41c4274223f5061212990e7416886c7d1beffc7...,2025-03-02 06:25:05.958118


## Step 3: Register feature definitions and deploy your feature store

`feast apply` scans python files in the current directory for feature/entity definitions and deploys infrastructure according to `feature_store.yaml`.

### Step 3a: Inspecting feature definitions
Let's inspect what `example_repo.py` looks like:

```python
from datetime import timedelta

from feast import (
    FeatureView,
    Field,
    FileSource,
)
from feast.data_format import ParquetFormat
from feast.types import Float32, Array, String, ValueType
from feast import Entity

chunk = Entity(
    name="chunk_id",
    description="Chunk ID",
    value_type=ValueType.STRING,
)

parquet_file_path = "./data/docling_samples.parquet"

source = FileSource(
    file_format=ParquetFormat(),
    path=parquet_file_path,
    timestamp_field="created",
)

city_embeddings_feature_view = FeatureView(
    name="docling_fv",
    entities=[chunk],
    schema=[
        Field(name="file_name", dtype=String),
        Field(name="full_document_markdown", dtype=String),
        Field(name="raw_chunk_markdown", dtype=String),
        Field(
            name="chunk_embedding",
            dtype=Array(Float32),
            vector_index=True,
            vector_search_metric="COSINE",
        ),
        Field(name="bytes", dtype=String),
        Field(name="chunk_id", dtype=String),
    ],
    source=source,
    ttl=timedelta(hours=2),
)
```

### Step 3b: Applying feature definitions
Now we run `feast apply` to register the feature views and entities defined in `example_repo.py`, and sets up SQLite online store tables. Note that we had previously specified SQLite as the online store in `feature_store.yaml` by specifying a `local` provider.

In [26]:
%rm -rf .ipynb_checkpoints/

In [27]:
! feast apply 

/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pymilvus/client/__init__.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import DistributionNotFound, get_distribution
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/environs/__init__.py:58: DeprecationWarning: The '__version_info__' attribute is deprecated and will be removed in in a future version. Use feature detection or 'packaging.Version(importlib.metadata.version("marshmallow")).release' instead.
  _SUP

## Step 5: Load features into your online store

In [8]:
from datetime import datetime
from feast import FeatureStore

store = FeatureStore(repo_path=".")

/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pymilvus/client/__init__.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import DistributionNotFound, get_distribution
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/pkg_resources/__init__.py:3142: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/farceo/dev/feast/.venv/lib/python3.11/site-packages/environs/__init__.py:58: DeprecationWarning: The '__version_info__' attribute is deprecated and will be removed in in a future version. Use feature detection or 'packaging.Version(importlib.metadata.version("marshmallow")).release' instead.
  _SUP

### Step 5a: Using `write_to_online_store`

We now serialize the latest values of features since the beginning of time to prepare for serving. Note, `materialize_incremental` serializes all new features since the last `materialize` call, or since the time provided minus the `ttl` timedelta. In this case, this will be `CURRENT_TIME - 1 day` (`ttl` was set on the `FeatureView` instances in [feature_repo/feature_repo/example_repo.py](feature_repo/feature_repo/example_repo.py)). 

```bash
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME
```

An alternative to using the CLI command is to use Python:

In [12]:
store.write_to_online_store(feature_view_name='docling_feature_view', df=df)

### Step 5b: Inspect materialized features

Note that now there are `online_store.db` and `registry.db`, which store the materialized features and schema information, respectively.

In [14]:
pymilvus_client = store._provider._online_store._connect(store.config)
COLLECTION_NAME = pymilvus_client.list_collections()[0]

milvus_query_result = pymilvus_client.query(
    collection_name=COLLECTION_NAME,
    filter="file_name == '2203.01017v2'",
)
pd.DataFrame(milvus_query_result[0]).head()

,chunk_id_pk,chunk_embedding,chunk_id,created_ts,event_ts,file_name,raw_chunk_markdown
0,0100000002000000080000006368756e6b5f6964020000...,0.051321,002bce0097246931724ae35b1e1a0d13fbb2c1a97e6c04...,0,1740914705958118,2203.01017v2,a. Picture of a table:\n95% on complex tables.
1,0100000002000000080000006368756e6b5f6964020000...,0.091583,002bce0097246931724ae35b1e1a0d13fbb2c1a97e6c04...,0,1740914705958118,2203.01017v2,a. Picture of a table:\n95% on complex tables.
2,0100000002000000080000006368756e6b5f6964020000...,-0.039993,002bce0097246931724ae35b1e1a0d13fbb2c1a97e6c04...,0,1740914705958118,2203.01017v2,a. Picture of a table:\n95% on complex tables.
3,0100000002000000080000006368756e6b5f6964020000...,0.028728,002bce0097246931724ae35b1e1a0d13fbb2c1a97e6c04...,0,1740914705958118,2203.01017v2,a. Picture of a table:\n95% on complex tables.
4,0100000002000000080000006368756e6b5f6964020000...,-0.003588,002bce0097246931724ae35b1e1a0d13fbb2c1a97e6c04...,0,1740914705958118,2203.01017v2,a. Picture of a table:\n95% on complex tables.


### Quick note on entity keys
Note from the above command that the online store indexes by `entity_key`. 

[Entity keys](https://docs.feast.dev/getting-started/concepts/entity#entity-key) include a list of all entities needed (e.g. all relevant primary keys) to generate the feature vector. In this case, this is a serialized version of the `driver_id`. We use this later to fetch all features for a given driver at inference time.

## Step 6: Embedding a query using PyTorch and Sentence Transformers

During inference (e.g., during when a user submits a chat message) we need to embed the input text. This can be thought of as a feature transformation of the input data. In this example, we'll do this with a small Sentence Transformer from Hugging Face.

In [9]:
import torch
import torch.nn.functional as F
from feast import FeatureStore
from pymilvus import MilvusClient, DataType, FieldSchema
from transformers import AutoTokenizer, AutoModel
from example_repo import city_embeddings_feature_view, item

TOKENIZER = "sentence-transformers/all-MiniLM-L6-v2"
MODEL = "sentence-transformers/all-MiniLM-L6-v2"

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

def run_model(sentences, tokenizer, model):
    encoded_input = tokenizer(
        sentences, padding=True, truncation=True, return_tensors="pt"
    )
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings

## Step 7: Fetching real-time vectors and data for online inference

At inference time, we need to use vector similarity search through the document embeddings from the online feature store using `retrieve_online_documents_v2()` while passing the embedded query. These feature vectors can then be fed into the context of the LLM.

In [10]:
question = "Which city has the largest population in New York?"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
model = AutoModel.from_pretrained(MODEL)
query_embedding = run_model(question, tokenizer, model)
query = query_embedding.detach().cpu().numpy().tolist()[0]

In [11]:
from IPython.display import display

# Retrieve top k documents
context_data = store.retrieve_online_documents_v2(
    features=[
        "city_embeddings:vector",
        "city_embeddings:item_id",
        "city_embeddings:state",
        "city_embeddings:sentence_chunks",
        "city_embeddings:wiki_summary",
    ],
    query=query,
    top_k=3,
    distance_metric='COSINE',
).to_df()
display(context_data)

,vector,item_id,state,sentence_chunks,wiki_summary,distance
0,"[0.15548758208751678, -0.08017724752426147, -0...",0,"New York, New York","New York, often called New York City or simply...","New York, often called New York City or simply...",0.743023


In [12]:
def format_documents(context_df):
    output_context = ""
    unique_documents = context_df.drop_duplicates().apply(
        lambda x: "City & State = {" + x['state'] +"}\nSummary = {" + x['wiki_summary'].strip()+"}",
        axis=1,
    )
    for i, document_text in enumerate(unique_documents):
        output_context+= f"****START DOCUMENT {i}****\n{document_text.strip()}\n****END DOCUMENT {i}****"
    return output_context

In [13]:
RAG_CONTEXT = format_documents(context_data[['state', 'wiki_summary']])

In [14]:
print(RAG_CONTEXT)

****START DOCUMENT 0****
City & State = {New York, New York}
Summary = {New York, often called New York City or simply NYC, is the most populous city in the United States, located at the southern tip of New York State on one of the world's largest natural harbors. The city comprises five boroughs, each of which is coextensive with a respective county. New York is a global center of finance and commerce, culture and technology, entertainment and media, academics and scientific output, and the arts and fashion, and, as home to the headquarters of the United Nations, is an important center for international diplomacy. New York City is the epicenter of the world's principal metropolitan economy.
With an estimated population in 2022 of 8,335,897 distributed over 300.46 square miles (778.2 km2), the city is the most densely populated major city in the United States. New York has more than double the population of Los Angeles, the nation's second-most populous city. New York is the geographic

In [15]:
FULL_PROMPT = f"""
You are an assistant for answering questions about states. You will be provided documentation from Wikipedia. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.

Here are document(s) you should use when answer the users question:
{RAG_CONTEXT}
"""

In [16]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [17]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": FULL_PROMPT},
        {"role": "user", "content": question}
    ],
)

In [18]:
print('\n'.join([c.message.content for c in response.choices]))

The largest city in New York is New York City, often referred to as NYC. It is the most populous city in the United States, with an estimated population of 8,335,897 in 2022.


# End